In [95]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

def extract_frames(video_path, output_folder, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_frames if total_frames > num_frames else 1

    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    except OSError:
        print(f"Error: Creating directory {output_folder}")

    frame_ids = [int(interval * i) for i in range(num_frames)]
    frame_count = 0
    saved_frames = 0

    while saved_frames < num_frames:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_ids[saved_frames])
        success, frame = cap.read()

        if not success:
            print(f"Failed to read frame at index {frame_ids[saved_frames]} from {video_path}")
            saved_frames += 1
            continue

        if frame is None or frame.size == 0:
            print(f"Warning: Empty or corrupt frame at index {frame_ids[saved_frames]} in {video_path}")
            saved_frames += 1
            continue

        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        saved_frames += 1
        frame_count = frame_ids[saved_frames] if saved_frames < num_frames else frame_count

    cap.release()
    cv2.destroyAllWindows()



def load_and_preprocess_data(root_folder):
    images = []
    labels = []
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

    # Iterate through each player's folder to load images
    for player_name in os.listdir(root_folder):
        player_folder = os.path.join(root_folder, player_name)
        if os.path.isdir(player_folder):
            for img_file in os.listdir(player_folder):
                if img_file.lower().endswith(valid_extensions):
                    img_path = os.path.join(player_folder, img_file)
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Warning: Could not read image {img_path} - it may be corrupt or in an unsupported format.")
                        continue

                    img = cv2.resize(img, (64, 64))
                    images.append(img)
                    labels.append(player_name)  # Label is the player's name

    images = np.array(images)
    labels = np.array(labels)

    # Encode labels to integers
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    categorical_labels = to_categorical(encoded_labels)  # One-hot encoding

    return images, categorical_labels, encoder

def build_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def main():
    output_frame_folder = '/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/Hybrid_Spatio_Temporal_Model_For_Gait_Analysis/extracted_frames'

    X_train, y_train, encoder = load_and_preprocess_data(output_frame_folder)
    model = build_model(len(encoder.classes_))
    model.fit(X_train, y_train, epochs=10)

    model.save('player_recognition_model.h5')
    np.save('label_encoder_classes.npy', encoder.classes_)

if __name__ == "__main__":
    main()

Epoch 1/10
20/20 [==============================] - 1s 30ms/step - loss: 65.2288 - accuracy: 0.2041
Epoch 2/10
20/20 [==============================] - 1s 28ms/step - loss: 1.1610 - accuracy: 0.6029
Epoch 3/10
20/20 [==============================] - 1s 28ms/step - loss: 0.4360 - accuracy: 0.8612
Epoch 4/10
20/20 [==============================] - 1s 28ms/step - loss: 0.2101 - accuracy: 0.9410
Epoch 5/10
20/20 [==============================] - 1s 32ms/step - loss: 0.1057 - accuracy: 0.9777
Epoch 6/10
20/20 [==============================] - 1s 28ms/step - loss: 0.1950 - accuracy: 0.9681
Epoch 7/10
20/20 [==============================] - 1s 28ms/step - loss: 0.1641 - accuracy: 0.9633
Epoch 8/10
20/20 [==============================] - 1s 28ms/step - loss: 0.1573 - accuracy: 0.9745
Epoch 9/10
20/20 [==============================] - 1s 28ms/step - loss: 0.1094 - accuracy: 0.9777
Epoch 10/10
20/20 [==============================] - 1s 30ms/step - loss: 0.1831 - accuracy: 0.9681


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [101]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Function to extract frames
def extract_frames_for_prediction(video_path, output_folder, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_frames
    frame_ids = [int(interval * i) for i in range(num_frames)]
    frames = []

    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    except OSError:
        print(f"Error: Creating directory {output_folder}")

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count in frame_ids:
            resized_frame = cv2.resize(frame, (64, 64))
            frames.append(resized_frame)
        frame_count += 1

    cap.release()
    return np.array(frames)

# Load the model and label encoder
model = load_model('player_recognition_model.h5')
encoder = LabelEncoder()
encoder.classes_ = np.load('label_encoder_classes.npy')

# Prediction function
def predict_player(video_path):
    frame_folder = 'temp_frames'
    frames = extract_frames_for_prediction(video_path, frame_folder)
    predictions = model.predict(frames)
    predicted_class = np.argmax(np.mean(predictions, axis=0))
    predicted_player = encoder.inverse_transform([predicted_class])[0]  # Translate label index back to player name

    return predicted_player

# Example usage
video_path = '/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/Datasets/untitled folder/1.mov'  # Path to your test video clip
result = predict_player(video_path)
print(f"The player in the video is: {result}")


1/1 [==============================] - 0s 34ms/step
The player in the video is: Hardik_Pandya
